<Response [200]>

In [8]:
import pandas as pd
import time
import datetime
import csv

In [9]:
def getData(team, after, before):
    url = 'https://api.pushshift.io/reddit/search/submission/?q='+str(team)+'&subreddit=PremierLeague&after='+str(after)+'&before='+str(before)+'&size=100'
    epl = requests.get(url)
    push = epl.json()
    return push['data']

In [10]:
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec

In [12]:
text = []
team = ['Arsenal', 'Aston Villa', 'Burnley', 'Chelsea', 'Crystal Palace',
'Everton', 'Hull City', 'Leicester City', 'Liverpool',
'Manchester City', 'Manchester United', 'Newcastle', 'QPR', 'Southampton', 'Stoke City',
'Sunderland', 'Swansea', 'Tottenham', 'WBA', 'West Ham']

# Gathering all the text datas and train them by Word2Vec

for t in team:
    after = 1299164400 # From the beginning of the subreddit created
    before = 1595862000 # End of the 2019-2020 seasons
    data = getData(t, after, before)
    while len(data) > 0:
        for submission in data:
            if 'selftext' in submission:
                text.append(submission['selftext'])
        after = data[-1]['created_utc']
        data = getData(t, after, before)
lists = []
for i in range(len(text)):
    lists.append(simple_preprocess(text[i]))
model = Word2Vec(lists, vector_size=100, window=5, min_count=5, workers=4, sg = 0)

In [13]:
good = ['good', 'positive', 'best', 'amazing', 'win', 
        'nice', 'love', 'fun', 'favorite', 
        'cool', 'impressive', 'great']
bad = ['bad', 'awful', 'negative', 'lose', 'terrible',
      'worst', 'boring', 'hate', 'unimpressive', 'frustrated',
      'struggle', 'pesty']

In [14]:
# Get 30 top similar words of good and bad word lists

g = []
for i in good:
    g.append(model.wv.most_similar(i, topn = 30))
b = []
for j in bad:

    b.append(model.wv.most_similar(j, topn = 30))

In [26]:
# stores only words

p = []
for i in g:
    for j in i:
        p.append(j[0])
        
n = []
for a in b:
    for b in a:
        n.append(b[0])

In [22]:
# Get all the features and store them in subStats dictionary

subStats = dict()
season_start = [1376665200, 1408114800, 1438959600, 1471014000,
               1502377200, 1533913200, 1565276400, 1595862000]
season_name = ['2012-2013','2013-2014','2014-2015','2015-2016',
               '2016-2017','2017-2018','2018-2019','2019-2020']

for t in team:
    after = 1345215600 
    before = 1595862000 
    numComms = 0
    score = 0
    subCount = 0
    count = 0
    data = getData(t, after, before)
    decide = 0
    up = 0
    down = 0
    while len(data) > 0:
        for submission in data:
            if submission['created_utc'] > season_start[count]:
                name = str(t)+str(season_name[count])
                subStats[name]=[season_name[count], t, subCount, score, numComms, up, down]
                numComms = 0
                score = 0
                subCount = 0
                up = 0
                down = 0
                count += 1
            if 'selftext' in submission:
                text.append(submission['selftext'])
                for e in p:
                    if e in simple_preprocess(submission['selftext']):
                        decide += 1
                for l in n:
                    if l in simple_preprocess(submission['selftext']):
                        decide -= 1
            if decide > 0:
                up += 1
                decide = 0
            elif decide < 0:
                down += 1
                decide = 0
            numComms+=submission['num_comments']
            score += submission['score']
            subCount+=1
        after = data[-1]['created_utc']
        data = getData(t, after, before)
    name = str(t)+str(season_name[count])
    subStats[name]=[season_name[count], t, subCount, score, numComms, up, down]

1376746682
Arsenal2012-2013
1408130176
Arsenal2013-2014
1438983147
Arsenal2014-2015
1471190149
Arsenal2015-2016
1502478215
Arsenal2016-2017
1533988219
Arsenal2017-2018
1565279931
Arsenal2018-2019
Arsenal2019-2020
Arsenal
1376746682
Aston Villa2012-2013
1408122065
Aston Villa2013-2014
1439506465
Aston Villa2014-2015
1471257070
Aston Villa2015-2016
1504957755
Aston Villa2016-2017
1540756296
Aston Villa2017-2018
1565324942
Aston Villa2018-2019
Aston Villa2019-2020
Aston Villa
1389300203
Burnley2012-2013
1408175389
Burnley2013-2014
1454165721
Burnley2014-2015
1471455940
Burnley2015-2016
1502551692
Burnley2016-2017
1533988219
Burnley2017-2018
1565386090
Burnley2018-2019
Burnley2019-2020
Burnley
1376746682
Chelsea2012-2013
1408175389
Chelsea2013-2014
1438983147
Chelsea2014-2015
1471222877
Chelsea2015-2016
1502551692
Chelsea2016-2017
1533922258
Chelsea2017-2018
1565291112
Chelsea2018-2019
Chelsea2019-2020
Chelsea
1376746682
Crystal Palace2012-2013
1408204693
Crystal Palace2013-2014
1439583152

In [24]:
# create csv file

file = "/Users/bradyhong/Downloads/epl.csv"
with open(file, 'w', newline = '', encoding='utf-8') as file:
    a = csv.writer(file, delimiter=',')
    headers = ["Season", "Team", "Total Post", "Total Comments", "Score", "Postivie", "Negative"]
    a.writerow(headers)
    for sub in subStats:
        a.writerow(subStats[sub])